In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!nvidia-smi

Thu Sep 22 13:41:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import gc
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import re
import html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam, SGD, AdamW, RAdam
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold,GroupKFold
from sklearn.metrics import log_loss,f1_score

from transformers import AutoModel, AutoConfig, AutoTokenizer, AdamW, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

In [5]:
DIR = '/content/drive/MyDrive/Competitions/Signate/MUFJ'
INPUT_DIR = os.path.join(DIR,'input')
OUTPUT_DIR = os.path.join(DIR,'output')
OUTPUT_SUB_DIR = os.path.join(OUTPUT_DIR,'submission')
#OUTPUT_MODEL_DIR = os.path.join(OUTPUT_DIR,'model')
#OUTPUT_MODEL_DIR19 = DIR + '/output/model/EXP19/'
OUTPUT_MODEL_DIR20 = DIR + '/output/model/EXP20/'
#OUTPUT_MODEL_DIR22 = DIR + '/output/model/EXP22/'
#OUTPUT_MODEL_DIR23 = DIR + '/output/model/EXP23/'
#OUTPUT_MODEL_DIR24 = DIR + '/output/model/EXP24/'
#OUTPUT_MODEL_DIR25 = DIR + '/output/model/EXP25/'
#OUTPUT_MODEL_DIR27 = DIR + '/output/model/EXP27/'
#OUTPUT_MODEL_DIR28 = DIR + '/output/model/EXP28/'
#OUTPUT_MODEL_DIR31 = DIR + '/output/model/EXP31/'
#OUTPUT_MODEL_DIR41 = DIR + '/output/model/EXP41/'
OUTPUT_MODEL_DIR42 = DIR + '/output/model/EXP42/'

#OUTPUT_MODEL_DIR21 = DIR + '/output/model/EXP21/'
#OUTPUT_MODEL_DIR43 = DIR + '/output/model/EXP43/'
OUTPUT_MODEL_DIR44 = DIR + '/output/model/EXP44/'
OUTPUT_MODEL_DIR45 = DIR + '/output/model/EXP45/'
#OUTPUT_MODEL_DIR46 = DIR + '/output/model/EXP46/'

OUTPUT_MODEL_DIR34 = DIR + '/output/model/EXP34/'
#OUTPUT_MODEL_DIR39 = DIR + '/output/model/EXP39/'
#OUTPUT_MODEL_DIR40 = DIR + '/output/model/EXP40/'
OUTPUT_MODEL_DIR47 = DIR + '/output/model/EXP47/'
OUTPUT_MODEL_DIR48 = DIR + '/output/model/EXP48/'

In [6]:
class CFG20:
    num_workers=4
    path=OUTPUT_MODEL_DIR20
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=True
    freezing=True
    clean_content = True
    target2col = False



class CFG42:
    num_workers=4
    path=OUTPUT_MODEL_DIR42
    config_path=path+'config.pth'
    model="facebook/bart-large-mnli"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=180
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = True
    target2col = False



class CFG44:
    num_workers=4
    path=OUTPUT_MODEL_DIR44
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-large"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=True
    clean_content = True
    target2col = False

class CFG45:
    num_workers=4
    path=OUTPUT_MODEL_DIR45
    config_path=path+'config.pth'
    model="facebook/bart-large-mnli"
    batch_size=32
    fc_dropout=0.2
    target_size=1
    max_len=180
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = False
    target2col = False



class CFG34:
    num_workers=4
    path=OUTPUT_MODEL_DIR34
    config_path=path+'config.pth'
    model="microsoft/deberta-v3-base"
    batch_size=32
    fc_dropout=0.2
    target_size=2
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=True
    freezing=True
    clean_content = True
    target_cols = ["state","category1"]
    target2col = True



class CFG47:
    num_workers=4
    path=OUTPUT_MODEL_DIR47
    config_path=path+'config.pth'
    model="funnel-transformer/large"
    batch_size=32
    fc_dropout=0.2
    target_size=2
    max_len=256
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = True
    target_cols = ["state","category1"]
    target2col = True

class CFG48:
    num_workers=4
    path=OUTPUT_MODEL_DIR48
    config_path=path+'config.pth'
    model="facebook/bart-large-mnli"
    batch_size=32
    fc_dropout=0.2
    target_size=2
    max_len=180
    seed=42
    n_fold=4
    trn_fold=[1, 2, 3]
    gradient_checkpointing=False
    freezing=False
    clean_content = True
    target_cols = ["state","category1"]
    target2col = True

In [7]:
def get_score(labels, outputs):
    thresh = 0.5
    y_pred = outputs
    y_true = labels
    return f1_score(y_true, (y_pred>thresh).astype(int))


def get_logger(filename=OUTPUT_DIR+'/infrence'):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=seed)

In [8]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
        
def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)
            
    return freezed_parameters

def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )

In [9]:
#oof_19 = pd.read_pickle(CFG19.path+'oof_df.pkl')
oof_20 = pd.read_pickle(CFG20.path+'oof_df.pkl')
#oof_22 = pd.read_pickle(CFG22.path+'oof_df.pkl')
#oof_23 = pd.read_pickle(CFG23.path+'oof_df.pkl')
#oof_24 = pd.read_pickle(CFG24.path+'oof_df.pkl')
#oof_25 = pd.read_pickle(CFG25.path+'oof_df.pkl')
#oof_27 = pd.read_pickle(CFG27.path+'oof_df.pkl')
#oof_28 = pd.read_pickle(CFG28.path+'oof_df.pkl')
#oof_31 = pd.read_pickle(CFG31.path+'oof_df.pkl')
#oof_41 = pd.read_pickle(CFG41.path+'oof_df.pkl')
oof_42 = pd.read_pickle(CFG42.path+'oof_df.pkl')

#oof_21 = pd.read_pickle(CFG21.path+'oof_df.pkl')
#oof_43 = pd.read_pickle(CFG43.path+'oof_df.pkl')
oof_44 = pd.read_pickle(CFG44.path+'oof_df.pkl')
oof_45 = pd.read_pickle(CFG45.path+'oof_df.pkl')
#oof_46 = pd.read_pickle(CFG46.path+'oof_df.pkl')

oof_34 = pd.read_pickle(CFG34.path+'oof_df.pkl')
#oof_39 = pd.read_pickle(CFG39.path+'oof_df.pkl')
#oof_40 = pd.read_pickle(CFG40.path+'oof_df.pkl')
oof_47 = pd.read_pickle(CFG47.path+'oof_df.pkl')
oof_48 = pd.read_pickle(CFG48.path+'oof_df.pkl')

In [10]:
labels = oof_20['state'].values

#preds_19 = oof_19['pred'].values
preds_20 = oof_20['pred'].values
#preds_22 = oof_22['pred'].values
#preds_23 = oof_23['pred'].values
#preds_24 = oof_24['pred'].values
#preds_25 = oof_25['pred'].values
#preds_27 = oof_27['pred'].values
#preds_28 = oof_28['pred'].values
#preds_31 = oof_31['pred'].values
#preds_41 = oof_41['pred'].values
preds_42 = oof_42['pred'].values

#preds_21 = oof_21['pred'].values
#preds_43 = oof_43['pred'].values
preds_44 = oof_44['pred'].values
preds_45 = oof_45['pred'].values
#preds_46 = oof_46['pred'].values

preds_34 = oof_34["pred_state"].values
#preds_39 = oof_39["pred_state"].values
#preds_40 = oof_40["pred_state"].values
preds_47 = oof_47["pred_state"].values
preds_48 = oof_48["pred_state"].values

In [11]:
preds = (preds_20 + preds_42 + preds_44 + preds_45 + preds_34 + preds_47 + preds_48) / 7
score = get_score(labels, preds)
print(f"F1 Score : {score}")

F1 Score : 0.8266397578203835


In [12]:
#best score: 0.8319967923015238
#best weight: [0.06470878 0.0131939  0.06180668 0.05055884 0.02008916 0.06263664
# 0.08175019 0.03084266 0.03858474 0.02707047 0.08370543 0.07534494
# 0.08425437 0.07985377 0.08337871 0.01404711 0.02728774 0.00701513
# 0.03316138 0.02549253 0.03521683]

#best score: 0.831077694235589
#best weight: [0.04754912 0.04911001 0.37500948 0.37216546 0.01848971 0.13767623]

# oof > 0.805
#best score: 0.8310099287935011
#best weight: [0.03209519 0.00845168 0.12410198 0.00022835 0.05227489 0.10028456
# 0.00259968 0.01994933 0.11993072 0.0957319  0.09343536 0.11866233
# 0.05481453 0.05538527 0.10995686 0.0027024  0.00939497]

# oof > 0.810
#best score: 0.8315239244069159
#best weight: [0.03682987 0.0061083  0.11077566 0.10054404 0.16015766 0.03514027
# 0.12992343 0.15352078 0.12774139 0.13925859]

In [13]:
test = pd.read_csv(os.path.join(INPUT_DIR,'test.csv'))
sub = pd.read_csv(os.path.join(INPUT_DIR,'sample_submit.csv'),header=None)
sub.columns = ['id','state']

In [14]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('',text)

def remove_html(text):
    html=re.compile(r"<[^>]*?>")
    return html.sub('',text)

def cleaning(texts):
    clean_texts = []
    for text in texts:
        # htmlタグを削除
        text = remove_URL(text)
        text = remove_html(text)
        #アルファベット以外をスペースに置き換え
        #clean_punc = re.sub(r'[^a-zA-Z]', ' ', text)
        #改行削除
        #text = text.replace("\n","")
        clean_texts.append(text)
    return clean_texts

def get_goal_values(df):
  df["goal"].replace("100000+","100000-100000",inplace=True)
  _df = df["goal"].str.split('-').apply(pd.Series).astype(float)
  _df.columns = ["goal_max","goal_min"]
  df["goal_max"] = _df["goal_max"].astype(str)
  df["goal_min"] = _df["goal_min"].astype(str)
  df["goal_median"] = _df[["goal_max","goal_min"]].median(axis=1)
  df["goal_median"] = df["goal_median"].astype(int)
  return df

if CFG20.clean_content==True:
    test['html_content'] = test['html_content'].map(lambda x: str(x))
    test['html_content'] = test['html_content'].apply(html.unescape)
    p = re.compile(r"<[^>]*?>|&amp;|[/'’\"”]")
    test['html_content'] = test['html_content'].map(lambda x: p.sub("", x))
    test['html_content'] = test['html_content'].map(lambda x: x.lstrip())
    test['html_content'] = test['html_content'].fillna('missing')

test = get_goal_values(test)
test['inputs1'] = test.goal_median.astype(str) + ' [SEP] ' + test.duration.astype(str) + ' [SEP] ' + test.country + ' [SEP] ' + test.category1 + ' [SEP] ' + test.category2 + ' [SEP] ' + test.html_content
test['inputs2'] = test.goal_median.astype(str) + ' [SEP] ' + test.duration.astype(str) + ' [SEP] ' + test.country + ' [SEP] ' + test.category2 + ' [SEP] ' + test.html_content

In [15]:
# ====================================================
# tokenizer
# ====================================================
#CFG19.tokenizer = AutoTokenizer.from_pretrained(CFG19.path+'tokenizer/')
CFG20.tokenizer = AutoTokenizer.from_pretrained(CFG20.path+'tokenizer/')
#CFG22.tokenizer = AutoTokenizer.from_pretrained(CFG22.path+'tokenizer/')
#CFG23.tokenizer = AutoTokenizer.from_pretrained(CFG23.path+'tokenizer/')
#CFG24.tokenizer = AutoTokenizer.from_pretrained(CFG24.path+'tokenizer/')
#CFG25.tokenizer = AutoTokenizer.from_pretrained(CFG25.path+'tokenizer/')
#CFG27.tokenizer = AutoTokenizer.from_pretrained(CFG27.path+'tokenizer/')
#CFG28.tokenizer = AutoTokenizer.from_pretrained(CFG28.path+'tokenizer/')
#CFG31.tokenizer = AutoTokenizer.from_pretrained(CFG31.path+'tokenizer/')
#CFG41.tokenizer = AutoTokenizer.from_pretrained(CFG41.path+'tokenizer/')
CFG42.tokenizer = AutoTokenizer.from_pretrained(CFG42.path+'tokenizer/')

#CFG21.tokenizer = AutoTokenizer.from_pretrained(CFG21.path+'tokenizer/')
#CFG43.tokenizer = AutoTokenizer.from_pretrained(CFG43.path+'tokenizer/')
CFG44.tokenizer = AutoTokenizer.from_pretrained(CFG44.path+'tokenizer/')
CFG45.tokenizer = AutoTokenizer.from_pretrained(CFG45.path+'tokenizer/')
#CFG46.tokenizer = AutoTokenizer.from_pretrained(CFG46.path+'tokenizer/')

CFG34.tokenizer = AutoTokenizer.from_pretrained(CFG34.path+'tokenizer/')
#CFG39.tokenizer = AutoTokenizer.from_pretrained(CFG39.path+'tokenizer/')
#CFG40.tokenizer = AutoTokenizer.from_pretrained(CFG40.path+'tokenizer/')
CFG47.tokenizer = AutoTokenizer.from_pretrained(CFG47.path+'tokenizer/')
CFG48.tokenizer = AutoTokenizer.from_pretrained(CFG48.path+'tokenizer/')

In [16]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False,
                           truncation=True)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs


class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        if cfg.target2col:
          self.inputs = df['inputs2'].values
        else:
          self.inputs = df['inputs1'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.inputs[item])
        return inputs

In [17]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class MaxPooling(nn.Module):
    def __init__(self):
        super(MaxPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        embeddings = last_hidden_state.clone()
        embeddings[input_mask_expanded == 0] = -1e4
        max_embeddings, _ = torch.max(embeddings, dim=1)
        return max_embeddings
    

class CustomModel1(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output


class CustomModel2(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        #self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

class CustomModel3(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        #self.pool = MeanPooling()
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

class CustomModel4(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())

        #self.pool = MeanPooling()
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        #self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        #self._init_weights(self.attention)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(self.fc_dropout(feature))
        return output

In [18]:
# ====================================================
# inference
# ====================================================
def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [19]:
test_dataset48 = TestDataset(CFG48, test)
test_loader48 = DataLoader(test_dataset48,
                         batch_size=CFG48.batch_size,
                         shuffle=False,
                         num_workers=CFG48.num_workers, pin_memory=True, drop_last=False)
predictions_48 = []
for fold in CFG48.trn_fold:
    model = CustomModel2(CFG48, config_path=CFG48.config_path, pretrained=False)
    state = torch.load(CFG48.path+f"{CFG48.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader48, model, device)
    predictions_48.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_48 = np.mean(predictions_48, axis=0)
predictions_48 = predictions_48[:,0]
predictions_48 = predictions_48.reshape((-1,1))


test_dataset20 = TestDataset(CFG20, test)
test_loader20 = DataLoader(test_dataset20,
                         batch_size=CFG20.batch_size,
                         shuffle=False,
                         num_workers=CFG20.num_workers, pin_memory=True, drop_last=False)
predictions_20 = []
for fold in CFG20.trn_fold:
    model = CustomModel1(CFG20, config_path=CFG20.config_path, pretrained=False)
    state = torch.load(CFG20.path+f"{CFG20.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader20, model, device)
    predictions_20.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_20 = np.mean(predictions_20, axis=0)


test_dataset42 = TestDataset(CFG42, test)
test_loader42 = DataLoader(test_dataset42,
                         batch_size=CFG42.batch_size,
                         shuffle=False,
                         num_workers=CFG42.num_workers, pin_memory=True, drop_last=False)
predictions_42 = []
for fold in CFG42.trn_fold:
    model = CustomModel2(CFG42, config_path=CFG42.config_path, pretrained=False)
    state = torch.load(CFG42.path+f"{CFG42.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader42, model, device)
    predictions_42.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_42 = np.mean(predictions_42, axis=0)


test_dataset44 = TestDataset(CFG44, test)
test_loader44 = DataLoader(test_dataset44,
                         batch_size=CFG44.batch_size,
                         shuffle=False,
                         num_workers=CFG44.num_workers, pin_memory=True, drop_last=False)
predictions_44 = []
for fold in CFG44.trn_fold:
    model = CustomModel3(CFG44, config_path=CFG44.config_path, pretrained=False)
    state = torch.load(CFG44.path+f"{CFG44.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader44, model, device)
    predictions_44.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_44 = np.mean(predictions_44, axis=0)


test_dataset45 = TestDataset(CFG45, test)
test_loader45 = DataLoader(test_dataset45,
                         batch_size=CFG45.batch_size,
                         shuffle=False,
                         num_workers=CFG45.num_workers, pin_memory=True, drop_last=False)
predictions_45 = []
for fold in CFG45.trn_fold:
    model = CustomModel4(CFG45, config_path=CFG45.config_path, pretrained=False)
    state = torch.load(CFG45.path+f"{CFG45.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader45, model, device)
    predictions_45.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_45 = np.mean(predictions_45, axis=0)


test_dataset34 = TestDataset(CFG34, test)
test_loader34 = DataLoader(test_dataset34,
                         batch_size=CFG34.batch_size,
                         shuffle=False,
                         num_workers=CFG34.num_workers, pin_memory=True, drop_last=False)
predictions_34 = []
for fold in CFG34.trn_fold:
    model = CustomModel1(CFG34, config_path=CFG34.config_path, pretrained=False)
    state = torch.load(CFG34.path+f"{CFG34.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader34, model, device)
    predictions_34.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_34 = np.mean(predictions_34, axis=0)
predictions_34 = predictions_34[:,0]
predictions_34 = predictions_34.reshape((-1,1))



test_dataset47 = TestDataset(CFG47, test)
test_loader47 = DataLoader(test_dataset47,
                         batch_size=CFG47.batch_size,
                         shuffle=False,
                         num_workers=CFG47.num_workers, pin_memory=True, drop_last=False)
predictions_47 = []
for fold in CFG47.trn_fold:
    model = CustomModel1(CFG47, config_path=CFG47.config_path, pretrained=False)
    state = torch.load(CFG47.path+f"{CFG47.model.replace('/', '-')}_fold{fold}_best.pth",
                       map_location=torch.device('cpu'))
    model.load_state_dict(state['model'])
    prediction = inference_fn(test_loader47, model, device)
    predictions_47.append(prediction)
    del model, state, prediction; gc.collect()
    torch.cuda.empty_cache()
predictions_47 = np.mean(predictions_47, axis=0)
predictions_47 = predictions_47[:,0]
predictions_47 = predictions_47.reshape((-1,1))




predictions = (predictions_20 + predictions_42 + predictions_44 + predictions_45 + predictions_34 + predictions_47 + predictions_48) /7

submit = sub.copy()
#submit.columns = ["id","label"]

submit['label'] = predictions
submit['state'] = (predictions>0.5).astype(int)

submit[["id","state"]].to_csv(os.path.join(OUTPUT_SUB_DIR,"exp108_ensemble_submission8266.csv"),index=False,header=False)
display(submit)

dup_test_ids = ['test_01704','test_03707','test_04353','test_04453','test_04645','test_06909','test_08232']
sub_dup= submit[submit["id"].isin(dup_test_ids)]

T = pd.DataFrame([['train_00365','train_01463','train_02156','train_04506','train_06891','train_08302','train_09470'],
                [0,1,1,1,1,1,0],
                ['test_08232','test_01704','test_06909','test_04453','test_03707','test_04353','test_04645']]).T
T.columns =["train id","true label","id"]
T = T.sort_values(by="id")
T = T.merge(sub_dup,how='left',on='id')
display(T)

plt.hist(submit.label,bins=25,edgecolor='white')
plt.xlabel("Pred")
plt.show()

 20%|█▉        | 61/307 [02:03<08:17,  2.02s/it]


KeyboardInterrupt: ignored